In [17]:
import kipoi

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
!pip install torchvision

     |████████████████████████████████| 1.0 MB 1.8 MB/s eta 0:00:01


In [4]:
!python AttentiveChrome/v2PyTorch/train.py --test_on_saved_model --save_attention_maps --kipoi_model --pgd \
--save_root ./Results_E016_fgsm --save_adv_inputs --only_run_model 'E016'

Traceback (most recent call last):
  File "AttentiveChrome/v2PyTorch/train.py", line 20, in <module>
    import data
  File "/Users/alex/Documents/prog/courses/compbio/Attention-CNNs-Genomics/AttentiveChrome/v2PyTorch/data.py", line 8, in <module>
    from torchvision import transforms, utils
ModuleNotFoundError: No module named 'torchvision'


In [1]:
import csv
import collections

import numpy as np


def loadData(filename,windows):
	with open(filename) as fi:
		csv_reader=csv.reader(fi)
		data=list(csv_reader)

		ncols=(len(data[0]))
	fi.close()
	nrows=len(data)
	ngenes=nrows/windows
	nfeatures=ncols-1
	print("Number of genes: %d" % ngenes)
	print("Number of entries: %d" % nrows)
	print("Number of HMs: %d" % nfeatures)

	count=0
	attr=collections.OrderedDict()
	Xs = []
	ys = []

	for i in range(0,nrows,windows):
		shape = (windows,)
		hm1=np.zeros(shape)
		hm2=np.zeros(shape)
		hm3=np.zeros(shape)
		hm4=np.zeros(shape)
		hm5=np.zeros(shape)
		for w in range(0,windows):
			if not i+w < nrows:
				break
			hm1[w]=int(data[i+w][2])
			hm2[w]=int(data[i+w][3])
			hm3[w]=int(data[i+w][4])
			hm4[w]=int(data[i+w][5])
			hm5[w]=int(data[i+w][6])
			
			thresholded_expr = int(data[i+w][7])

		Xs.append([hm1, hm2, hm3, hm4, hm5])
		ys.append(thresholded_expr)

	return np.array(Xs), np.array(ys)

def load_data(data_root, cell_type, n_bins=16, batch_size=16):
	'''
	Loads data into a 3D tensor for each of the 3 splits.

	'''
	print("==>loading train data")
	train=loadData(data_root+"/"+cell_type+"/classification/train.csv",n_bins)

	print("==>loading valid data")
	valid=loadData(data_root+"/"+cell_type+"/classification/valid.csv",n_bins)

	print("==>loading test data")
	test=loadData(data_root+"/"+cell_type+"/classification/test.csv",n_bins)

	return train,valid,test

#parser.add_argument('--data_root', type=str, default='./data/', help='data location')
#parser.add_argument('--cell_type', type=str, default='E003', help='cell type 1')

data = load_data('./AttentiveChrome/v2PyTorch/data', 'Toy')

==>loading train data
Number of genes: 62
Number of entries: 1000
Number of HMs: 7
==>loading valid data
Number of genes: 62
Number of entries: 1000
Number of HMs: 7
==>loading test data
Number of genes: 62
Number of entries: 1000
Number of HMs: 7


In [10]:
data[0][0].shape

(63, 5, 16)

In [43]:
!pip install tensorflow

[autoreload of numpy.core.overrides failed: Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/local/anaconda3/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/local/anaconda3/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/usr/local/anaconda3/lib/python3.8/site-packages/numpy/core/overrides.py", line 16, in <module>
    add_docstring(
RuntimeError: implement_array_function method already h

  Using cached tensorflow-2.4.0-cp38-cp38-macosx_10_11_x86_64.whl (175.5 MB)


In [23]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau

# https://github.com/titu1994/keras-attention-augmented-convs/blob/master/attn_augconv.py
# most of the implementation is taken from this repo
# I extended to add 1-D CNN implementation.
# Even though, it's (the extension) not tested properly yet.

from tensorflow.keras import initializers
from tensorflow.keras import backend as K

import tensorflow as tf

In [3]:
def _conv_layer(filters, kernel_size, strides=(1, 1), padding='same', name=None):
    return Conv2D(filters, kernel_size, strides=strides, padding=padding,
                  use_bias=True, kernel_initializer='he_normal', name=name)


def _conv_layer1d(ip, t_n, f_n, filters, kernel_size, strides=1, padding='same', name=None):
    

    conv1 = Conv1D(filters, kernel_size, strides=strides, padding=padding,
                  use_bias=True, kernel_initializer='he_normal', name=name)(ip)
    
    reshape = Reshape((t_n, 1, filters))(conv1)
    
    return reshape



def _conv_layer1r(ip, t_n, f_n, filters, kernel_size, strides=1, padding='same', name=None):
    
    reshape1 = Reshape((t_n, f_n))(ip)
    

    conv1 = Conv1D(filters, kernel_size, strides=strides, padding=padding,
                  use_bias=True, kernel_initializer='he_normal', name=name)(reshape1)
    
    reshape2 = Reshape((t_n, 1, filters))(conv1)
    
    return reshape2


def _normalize_depth_vars(depth_k, depth_v, filters):
    """
    Accepts depth_k and depth_v as either floats or integers
    and normalizes them to integers.
    Args:
        depth_k: float or int.
        depth_v: float or int.
        filters: number of output filters.
    Returns:
        depth_k, depth_v as integers.
    """

    if type(depth_k) == float:
        depth_k = int(filters * depth_k)
    else:
        depth_k = int(depth_k)

    if type(depth_v) == float:
        depth_v = int(filters * depth_v)
    else:
        depth_v = int(depth_v)

    return depth_k, depth_v


class AttentionAugmentation2D(Layer):

    def __init__(self, depth_k, depth_v, num_heads, relative=True, **kwargs):
        """
        Applies attention augmentation on a convolutional layer
        output.
        Args:
            depth_k: float or int. Number of filters for k.
            Computes the number of filters for `v`.
            If passed as float, computed as `filters * depth_k`.
        depth_v: float or int. Number of filters for v.
            Computes the number of filters for `k`.
            If passed as float, computed as `filters * depth_v`.
        num_heads: int. Number of attention heads.
            Must be set such that `depth_k // num_heads` is > 0.
        relative: bool, whether to use relative encodings.

        Raises:
            ValueError: if depth_v or depth_k is not divisible by
                num_heads.
        Returns:
            Output tensor of shape
            -   [Batch, Height, Width, Depth_V] if
                channels_last data format.
            -   [Batch, Depth_V, Height, Width] if
                channels_first data format.
        """
        super(AttentionAugmentation2D, self).__init__(**kwargs)

        if depth_k % num_heads != 0:
            raise ValueError('`depth_k` (%d) is not divisible by `num_heads` (%d)' % (
                depth_k, num_heads))

        if depth_v % num_heads != 0:
            raise ValueError('`depth_v` (%d) is not divisible by `num_heads` (%d)' % (
                depth_v, num_heads))

        if depth_k // num_heads < 1.:
            raise ValueError('depth_k / num_heads cannot be less than 1 ! '
                             'Given depth_k = %d, num_heads = %d' % (
                             depth_k, num_heads))

        if depth_v // num_heads < 1.:
            raise ValueError('depth_v / num_heads cannot be less than 1 ! '
                             'Given depth_v = %d, num_heads = %d' % (
                                 depth_v, num_heads))

        self.depth_k = depth_k
        self.depth_v = depth_v
        self.num_heads = num_heads
        self.relative = relative

        self.axis = 1 if K.image_data_format() == 'channels_first' else -1

    def build(self, input_shape):
        self._shape = input_shape

        # normalize the format of depth_v and depth_k
        self.depth_k, self.depth_v = _normalize_depth_vars(self.depth_k, self.depth_v,
                                                           input_shape)

        if self.axis == 1:
            _, channels, height, width = input_shape
        else:
            _, height, width, channels = input_shape

        if self.relative:
            dk_per_head = self.depth_k // self.num_heads
            
            # print(dk_per_head)

            if dk_per_head == 0:
                print('dk per head', dk_per_head)

            self.key_relative_w = self.add_weight('key_rel_w',
                                                  shape=tf.TensorShape([2 * width - 1, dk_per_head]),
                                                  initializer=initializers.RandomNormal(stddev=dk_per_head ** -0.5))
            # 2 * width - 1

            self.key_relative_h = self.add_weight('key_rel_h',
                                                  shape=tf.TensorShape([2 * height - 1, dk_per_head]),
                                                  initializer=initializers.RandomNormal(stddev=dk_per_head ** -0.5))
            # 2 * height - 1

        else:
            self.key_relative_w = None
            self.key_relative_h = None

    def call(self, inputs, **kwargs):
        if self.axis == 1:
            # If channels first, force it to be channels last for these ops
            inputs = K.permute_dimensions(inputs, [0, 2, 3, 1])

        q, k, v = tf.split(inputs, [self.depth_k, self.depth_k, self.depth_v], axis=-1)

        q = self.split_heads_2d(q)
        k = self.split_heads_2d(k)
        v = self.split_heads_2d(v)

        # scale query
        depth_k_heads = self.depth_k / self.num_heads
        q *= (depth_k_heads ** -0.5)

        # [Batch, num_heads, height * width, depth_k or depth_v] if axis == -1
        qk_shape = [self._batch, self.num_heads, self._height * self._width, self.depth_k // self.num_heads]
        v_shape = [self._batch, self.num_heads, self._height * self._width, self.depth_v // self.num_heads]
        flat_q = K.reshape(q, K.stack(qk_shape))
        flat_k = K.reshape(k, K.stack(qk_shape))
        flat_v = K.reshape(v, K.stack(v_shape))

        # [Batch, num_heads, HW, HW]
        logits = tf.matmul(flat_q, flat_k, transpose_b=True)

        # Apply relative encodings
        if self.relative:
            h_rel_logits, w_rel_logits = self.relative_logits(q)
            logits += h_rel_logits
            logits += w_rel_logits

        weights = K.softmax(logits, axis=-1)
        attn_out = tf.matmul(weights, flat_v)

        attn_out_shape = [self._batch, self.num_heads, self._height, self._width, self.depth_v // self.num_heads]
        attn_out_shape = K.stack(attn_out_shape)
        attn_out = K.reshape(attn_out, attn_out_shape)
        attn_out = self.combine_heads_2d(attn_out)
        # [batch, height, width, depth_v]

        if self.axis == 1:
            # return to [batch, depth_v, height, width] for channels first
            attn_out = K.permute_dimensions(attn_out, [0, 3, 1, 2])
        
        return attn_out

    def compute_output_shape(self, input_shape):
        output_shape = list(input_shape)
        output_shape[self.axis] = self.depth_v
        return tuple(output_shape)

    def split_heads_2d(self, ip):
        tensor_shape = K.shape(ip)

        # batch, height, width, channels for axis = -1
        tensor_shape = [tensor_shape[i] for i in range(len(self._shape))]

        batch = tensor_shape[0]
        height = tensor_shape[1]
        width = tensor_shape[2]
        channels = tensor_shape[3]

        # Save the spatial tensor dimensions
        self._batch = batch
        self._height = height
        self._width = width

        ret_shape = K.stack([batch, height, width,  self.num_heads, channels // self.num_heads])
        split = K.reshape(ip, ret_shape)
        transpose_axes = (0, 3, 1, 2, 4)
        split = K.permute_dimensions(split, transpose_axes)

        return split

    def relative_logits(self, q):
        shape = K.shape(q)
        # [batch, num_heads, H, W, depth_v]
        shape = [shape[i] for i in range(5)]

        height = shape[2]
        width = shape[3]

        rel_logits_w = self.relative_logits_1d(q, self.key_relative_w, height, width,
                                               transpose_mask=[0, 1, 2, 4, 3, 5])

        rel_logits_h = self.relative_logits_1d(
            K.permute_dimensions(q, [0, 1, 3, 2, 4]),
            self.key_relative_h, width, height,
            transpose_mask=[0, 1, 4, 2, 5, 3])

        return rel_logits_h, rel_logits_w

    def relative_logits_1d(self, q, rel_k, H, W, transpose_mask):
        rel_logits = tf.einsum('bhxyd,md->bhxym', q, rel_k)
        rel_logits = K.reshape(rel_logits, [-1, self.num_heads * H, W, 2 * W - 1])
        rel_logits = self.rel_to_abs(rel_logits)
        rel_logits = K.reshape(rel_logits, [-1, self.num_heads, H, W, W])
        rel_logits = K.expand_dims(rel_logits, axis=3)
        rel_logits = K.tile(rel_logits, [1, 1, 1, H, 1, 1])
        rel_logits = K.permute_dimensions(rel_logits, transpose_mask)
        rel_logits = K.reshape(rel_logits, [-1, self.num_heads, H * W, H * W])
        return rel_logits

    def rel_to_abs(self, x):
        shape = K.shape(x)
        shape = [shape[i] for i in range(3)]
        B, Nh, L, = shape
        col_pad = K.zeros(K.stack([B, Nh, L, 1]))
        x = K.concatenate([x, col_pad], axis=3)
        flat_x = K.reshape(x, [B, Nh, L * 2 * L])
        flat_pad = K.zeros(K.stack([B, Nh, L - 1]))
        flat_x_padded = K.concatenate([flat_x, flat_pad], axis=2)
        final_x = K.reshape(flat_x_padded, [B, Nh, L + 1, 2 * L - 1])
        final_x = final_x[:, :, :L, L - 1:]
        return final_x

    def combine_heads_2d(self, inputs):
        # [batch, num_heads, height, width, depth_v // num_heads]
        transposed = K.permute_dimensions(inputs, [0, 2, 3, 1, 4])
        # [batch, height, width, num_heads, depth_v // num_heads]
        shape = K.shape(transposed)
        shape = [shape[i] for i in range(5)]

        a, b = shape[-2:]
        ret_shape = K.stack(shape[:-2] + [a * b])
        # [batch, height, width, depth_v]
        return K.reshape(transposed, ret_shape)

    def get_config(self):
        config = {
            'depth_k': self.depth_k,
            'depth_v': self.depth_v,
            'num_heads': self.num_heads,
            'relative': self.relative,
        }
        base_config = super(AttentionAugmentation2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


def augmented_conv2d(ip, filters, kernel_size=(3, 3), strides=(1, 1),
                     depth_k=0.2, depth_v=0.2, num_heads=8, relative_encodings=True):
    """
    Builds an Attention Augmented Convolution block.
    Args:
        ip: keras tensor.
        filters: number of output filters.
        kernel_size: convolution kernel size.
        strides: strides of the convolution.
        depth_k: float or int. Number of filters for k.
            Computes the number of filters for `v`.
            If passed as float, computed as `filters * depth_k`.
        depth_v: float or int. Number of filters for v.
            Computes the number of filters for `k`.
            If passed as float, computed as `filters * depth_v`.
        num_heads: int. Number of attention heads.
            Must be set such that `depth_k // num_heads` is > 0.
        relative_encodings: bool. Whether to use relative
            encodings or not.
    Returns:
        a keras tensor.
    """
    # input_shape = K.int_shape(ip)
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1

    depth_k, depth_v = _normalize_depth_vars(depth_k, depth_v, filters)

    conv_out = _conv_layer(filters - depth_v, kernel_size, strides)(ip)

    # Augmented Attention Block
    qkv_conv = _conv_layer(2 * depth_k + depth_v, (1, 1), strides)(ip)
    attn_out = AttentionAugmentation2D(depth_k, depth_v, num_heads, relative_encodings)(qkv_conv)
    attn_out = _conv_layer(depth_v, kernel_size=(1, 1))(attn_out)

    output = concatenate([conv_out, attn_out], axis=channel_axis)
    return output

def augmented_conv1d(ip, shape, filters, kernel_size=3, strides=1, padding = 'same',
                     depth_k=0.2, depth_v=0.2, num_heads=2, relative_encodings=True):
    """
    Builds an Attention Augmented Convolution block.
    Args:
        ip: keras tensor.
        filters: number of output filters.
        kernel_size: convolution kernel size.
        strides: strides of the convolution.
        depth_k: float or int. Number of filters for k.
            Computes the number of filters for `v`.
            If passed as float, computed as `filters * depth_k`.
        depth_v: float or int. Number of filters for v.
            Computes the number of filters for `k`.
            If passed as float, computed as `filters * depth_v`.
        num_heads: int. Number of attention heads.
            Must be set such that `depth_k // num_heads` is > 0.
        relative_encodings: bool. Whether to use relative
            encodings or not.
    Returns:
        a keras tensor.
    """

    
    if type(kernel_size) == int:
        pass
    else:
        kernel_size = kernel_size[0]
        
    if type(strides) == int:
        pass
    else:
        strides = strides[0]
        
    t_n = shape[0]
    f_n = shape[1]
        
    # input_shape = K.int_shape(ip)
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1

    depth_k, depth_v = _normalize_depth_vars(depth_k, depth_v, filters)
    
    # print(kernel_size)
    # print(strides)

    conv_out = _conv_layer1d(ip, t_n, f_n, filters - depth_v, kernel_size, strides, padding = 'same')

    # Augmented Attention Block
    qkv_conv = _conv_layer1d(ip, t_n, f_n,  2 * depth_k + depth_v, 1, strides, padding = 'same')
    attn_out = AttentionAugmentation2D(depth_k, depth_v, num_heads, relative_encodings)(qkv_conv)
    attn_out = _conv_layer1r(attn_out, t_n, depth_v,  depth_v, 1, strides, padding = 'same')

    output = Concatenate(axis=channel_axis)([conv_out, attn_out])
   
    reshape = Reshape((t_n, filters))(output)

    return reshape

In [17]:
## Neural Network Settings 
## CNN Base Function
#@tf.function
def create_nn(N = 1, num_filters = 8, 
              dense_neurons = 8,
              conv_kernel_size = 5, 
              num_convolutions = 2,
              cnn_to_fc_op = "flatten",  #"flatten", "global_pool",
              dropout_rate = 0.4):
  
    inputShape = (N, 1)
    chanDim = -1

    # define the model input
    inputs = Input(shape=inputShape)
    x = inputs
    # x = BatchNormalization(axis=chanDim)(x)

    # # Perform multiple convolutional layers
    ## TODO: explore tuning of these options
    x = augmented_conv1d(x, shape = (N, 1), filters=num_filters, kernel_size=conv_kernel_size,
                     strides = 1,
                     padding = 'same', # if causal convolution is needed
                     depth_k=2, depth_v=2,  
                     num_heads=2, relative_encodings=True)
    #x = BatchNormalization(axis=chanDim)(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_rate)(x)

    x = MaxPooling1D(pool_size=2)(x)
    if cnn_to_fc_op == "flatten":
        x = Flatten()(x)
    elif cnn_to_fc_op == "global_pool":
        x = tf.keras.layers.GlobalAveragePooling1D()(x)
    else: 
        print("Use supported method for tensor decomposition")


    ## TODO: this is tunable
    x = Dense(dense_neurons, activation='relu')(x) 
    x = BatchNormalization(axis=chanDim)(x) 

    ## Output layer for binary classification will be one neuron (sigmoid act)
    x = Dense(1, activation='sigmoid')(x)

    # construct the NN
    model = Model(inputs, x)
    return model

## Neural Network Settings 

## Optimization settings:
num_epochs = 2000
batch_size = 512 
learning_rate = 1e-2
#lr_schedule = LearningRateScheduler(lr_scheduler)

## stop the model if no validation improvement in patience epochs
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='train_loss', 
                                                      min_delta=0.0001, 
                                                      patience=50)

## Directly Tunable architecture hyperparameters:
dropout_rate = 0.4
num_filters = 128
conv_kernel_size = 5
num_convolutions = 1
# recall N is the input length.  N*N_dense_multiplier for feed-forward layers
dense_neurons = 24

## logistics
#filepath = run_name+"model-{epoch:02d}.hdf5"
#checkpoint = ModelCheckpoint(filepath, verbose=0, save_best_only=False)

In [19]:
tf.compat.v1.disable_eager_execution()

In [26]:
ip = Input(shape=(N,1))

x = augmented_conv1d(ip, shape = (N, 1), filters=6, kernel_size=3,
                     strides = 1,
                     padding = 'same', # if causal convolution is needed
                     depth_k=2, depth_v=2,  
                     num_heads=2, relative_encodings=True)
x = Activation('relu')(x)
fl = Flatten()(x)
fc = Dense(1, activation='sigmoid')(fl)

aac_cnn = Model(ip, fc)

aac_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

aac_cnn.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 5, 1)]       0                                            
__________________________________________________________________________________________________
conv1d_18 (Conv1D)              (None, 5, 6)         12          input_4[0][0]                    
__________________________________________________________________________________________________
reshape_24 (Reshape)            (None, 5, 1, 6)      0           conv1d_18[0][0]                  
__________________________________________________________________________________________________
attention_augmentation2d_7 (Att (None, None, None, N 10          reshape_24[0][0]                 
____________________________________________________________________________________________

In [24]:
## Run the NN model
callbacks_list = [earlystop_callback]#checkpoint, earlystop_callback] #, tensorboard]#, reduce_lr]#, earlystopping]

N = 5

model = create_nn(N, 
                  num_filters, 
                  dense_neurons = dense_neurons,
                  conv_kernel_size = conv_kernel_size, 
                  num_convolutions = num_convolutions, 
                  cnn_to_fc_op = "flatten",
                  dropout_rate = dropout_rate)

opt = Adam(lr=learning_rate)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics = ['acc'], experimental_run_tf_function=False)#metrics=['acc', tf.keras.metrics.AUC()])
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 5, 1)]       0                                            
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, 5, 6)         12          input_3[0][0]                    
__________________________________________________________________________________________________
reshape_19 (Reshape)            (None, 5, 1, 6)      0           conv1d_15[0][0]                  
__________________________________________________________________________________________________
attention_augmentation2d_6 (Att (None, None, None, N 10          reshape_19[0][0]                 
______________________________________________________________________________________________

In [28]:
aac_hist = aac_cnn.fit(data[0][0], data[0][1], epochs = 2, batch_size = 16, validation_split = 0.5, verbose = 0)
aac_hist.history

AssertionError: 

In [ ]:
result = model.fit(xTrain, yTrain,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    shuffle=True,
                    verbose=1,
                    class_weight=class_weight,
                    validation_data= (xValidation, yValidation),
                    callbacks=callbacks_list)